# Código para aplicar a krigagem nos dados de tipos de solo

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import gstools as gs
import geopandas as gpd
import rioxarray as rxr

from time import sleep
from utils.consts import SOIL_TYPES
from soil_texture import calculate_soil_type
from skgstat import Variogram
from tqdm import tqdm, trange

In [2]:
dados    = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="Analise Solo")
uso_solo = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/USOSOLO.tif")

In [3]:
dados.dropna(inplace=True, ignore_index=True)

points = gpd.points_from_xy(dados["lat"], dados["lon"], crs="EPSG:4326")
dados = gpd.GeoDataFrame(dados[["Ponto", "PROF", "Argila", "Silte", "Areia(Grossa)", "Areia(Fina)"]], geometry=points, crs="EPSG:4326")
dados.to_crs("EPSG:31983", inplace=True)

dados

,Ponto,PROF,Argila,Silte,Areia(Grossa),Areia(Fina),geometry
0,02,0 - 2,27.0,3.0,58.0,12.0,POINT (592224.603 7787265.87)
1,02,0 - 20,41.0,9.0,37.0,13.0,POINT (592224.603 7787265.87)
2,02,60 - 80,42.0,5.0,37.0,16.0,POINT (592224.603 7787265.87)
3,04,0 - 2,11.0,12.0,49.0,28.0,POINT (592368.605 7790837.259)
4,04,0 - 20,13.0,19.0,53.0,15.0,POINT (592368.605 7790837.259)
...,...,...,...,...,...,...,...
196,161,"1,4m",8.0,21.0,31.0,40.0,POINT (595970.168 7791192.033)
197,162,0 - 2,14.0,31.0,37.0,18.0,POINT (595908.053 7791452.29)
198,162,0 - 20,21.0,16.0,42.0,21.0,POINT (595908.053 7791452.29)
199,162,60 - 80,14.0,41.0,23.0,22.0,POINT (595908.053 7791452.29)


In [4]:
profs = dados["PROF"].unique()
profs

array(['0 - 2', '0 - 20', '60 - 80', '40 - 60', '20 - 40', '0  - 20',
       '0 - 10', '30 - 50', '50 - 70', '40 - 50', '0 - 15', '1,4m',
       '1,6m'], dtype=object)

In [5]:
prof_02 = dados[dados["PROF"]=="0 - 2"]
prof_20 = dados[(dados["PROF"]=="0 - 20")|(dados["PROF"]=="0 - 10")|(dados["PROF"]=="0  - 20")|(dados["PROF"]=="0 - 15")]
prof_80 = dados[dados["PROF"].isin(["60 - 80", "40 - 60", "20 - 40", "30 - 50", "50 - 70", "40 - 50"])]

len(prof_02), len(prof_20), len(prof_80)

(51, 80, 68)

### Aplicando a Krigagem

In [6]:
gridx = uso_solo.x.values
gridy = uso_solo.y.values

gridx.shape, gridy.shape

((9099,), (9099,))

In [ ]:
tipos = ["Argila", "Silte", "Areia(Grossa)", "Areia(Fina)"]
profundidades = [
    2,
    20,
    80
]

In [ ]:
# Pega o template de coordenadas
template_coords = {"y": uso_solo.y, "x": uso_solo.x}

for prof in profundidades:
    if prof == 2:
        profs = prof_02
    elif prof == 20:
        profs = prof_20
    else:
        profs = prof_80
    
    x = profs.geometry.x
    y = profs.geometry.y
    coords = np.vstack([x, y]).T

    for tipo in tipos:
        print(f"Gerando {tipo} em {prof}cm")
        if (tipo=="Argila" and prof == 2) or (tipo=="Silte" and prof == 2) or (tipo=="Areia(Grossa)" and prof == 2):
            continue

        z = profs[tipo].values

        V = Variogram(
            coords, z,
            model='spherical',     # pode testar: 'exponential', 'gaussian'
            maxlag='median',       # distância máxima considerada
            n_lags=8               # número de "bins" do semivariograma
        )

        rang   = V.parameters[0]
        sill   = V.parameters[1]
        nugget = V.parameters[2]

        del V # Liberando memória
        sleep(2)

        model = gs.Spherical(dim=2, var=sill, len_scale=rang, nugget=nugget)
        ok = gs.krige.Ordinary(model, cond_pos=[x, y], cond_val=z)

        X, Y = np.meshgrid(gridx, gridy)
        field, var = ok((X, Y), chunk_size=7000000)   # field = valores preditos, var = variância

        del z, model, ok, X, Y # Liberando memória
        sleep(2)
        
        print(f"Salvando {tipo} em {prof}cm")
        ny, nx = len(gridy), len(gridx)
        field = field.reshape(ny, nx)
        var   = var.reshape(ny, nx)

        da_pred = xr.DataArray(
            field,
            coords=template_coords,
            dims=("y","x"),
            name="pred"
        )

        da_var = xr.DataArray(
            var,
            coords=template_coords,
            dims=("y","x"),
            name="var"
        )
        da_pred = da_pred.expand_dims("band").assign_coords(band=[1])
        da_var  = da_var.expand_dims("band").assign_coords(band=[2])
        stack = xr.concat([da_pred, da_var], dim="band")

        stack = stack.rio.write_crs(uso_solo.rio.crs)
        stack = stack.rio.write_transform(uso_solo.rio.transform())
        stack.rio.to_raster(fr"D:/Mestrado/Trabalho Final/SIG/{tipo}_{prof}.tif", compress="LZW")

        del field, var, da_pred, da_var, stack # Liberando memória
        sleep(2)

Gerando Argila em 20cm
Salvando Argila em 20cm
Gerando Silte em 20cm
Salvando Silte em 20cm
Gerando Areia(Grossa) em 20cm
Salvando Areia(Grossa) em 20cm
Gerando Areia(Fina) em 20cm
Salvando Areia(Fina) em 20cm
Gerando Argila em 80cm
Salvando Argila em 80cm
Gerando Silte em 80cm
Salvando Silte em 80cm
Gerando Areia(Grossa) em 80cm
Salvando Areia(Grossa) em 80cm
Gerando Areia(Fina) em 80cm
Salvando Areia(Fina) em 80cm


### A soma de areia silte e argila deve ser 100
- O código anterior pode gerar valores diferentes de 100

In [9]:
for prof in profundidades:
    print(f"Para {prof}cm:")

    argila = rxr.open_rasterio(fr"D:/Mestrado/Trabalho Final/SIG/Argila_{prof}.tif").values[0]
    silte  = rxr.open_rasterio(fr"D:/Mestrado/Trabalho Final/SIG/Silte_{prof}.tif").values[0]
    areiaf = rxr.open_rasterio(fr"D:/Mestrado/Trabalho Final/SIG/Areia(Grossa)_{prof}.tif").values[0]
    areiag = rxr.open_rasterio(fr"D:/Mestrado/Trabalho Final/SIG/Areia(Fina)_{prof}.tif").values[0]

    total = argila + silte + areiaf + areiag
    print("Total:", total.max(), total.min())

    total_safe = np.where(total == 0, 1, total)  # substitui 0 por 1 só para não dividir por 0

    argila_norm = (argila / total_safe) * 100
    silte_norm  = (silte  / total_safe) * 100
    areiaf_norm = (areiaf / total_safe) * 100
    areiag_norm = (areiag / total_safe) * 100

    check = argila_norm + silte_norm + areiaf_norm + areiag_norm
    print("New Total:", check.min(), check.max())  # deve estar bem próximo de 100

    # Pega o template de coordenadas
    coords = {"y": uso_solo.y, "x": uso_solo.x}

    for tipo in tipos:
        print(f"Salvando {tipo} para {prof}cm")

        field = argila_norm
        if tipo == "Silte":
            field = silte_norm
        elif tipo == "Areia(Grossa)":
            field = areiag_norm
        elif tipo == "Areia(Fina)":
            field = areiaf_norm

        da = xr.DataArray(
            field,
            coords=coords,
            dims=("y","x"),
            name="norm"
        )
        da = da.expand_dims("band").assign_coords(band=[1])

        stack = da.rio.write_crs(uso_solo.rio.crs)
        stack = da.rio.write_transform(uso_solo.rio.transform())
        stack.rio.to_raster(fr"D:/Mestrado/Trabalho Final/SIG/{tipo}_{prof}_norm.tif", compress="LZW")
    
    del argila, silte, areiaf, areiag, argila_norm, silte_norm, areiaf_norm, areiag_norm

Para 20cm:
Total: 114.81523534518257 78.9688561350393
New Total: 99.99999999999996 100.00000000000004
Salvando Argila para 20cm
Salvando Silte para 20cm
Salvando Areia(Grossa) para 20cm
Salvando Areia(Fina) para 20cm
Para 80cm:
Total: 120.82052580007004 79.15057409997718
New Total: 99.99999999999996 100.00000000000004
Salvando Argila para 80cm
Salvando Silte para 80cm
Salvando Areia(Grossa) para 80cm
Salvando Areia(Fina) para 80cm


### Texturas

In [8]:
for prof in [2, 20, 80]:
    print(f"Obtendo texturas para {prof}cm:")

    argila = rxr.open_rasterio(fr"D:/Mestrado/Trabalho Final/SIG/Argila_{prof}_norm.tif").values[0]
    silte  = rxr.open_rasterio(fr"D:/Mestrado/Trabalho Final/SIG/Silte_{prof}_norm.tif").values[0]
    areiaf = rxr.open_rasterio(fr"D:/Mestrado/Trabalho Final/SIG/Areia(Grossa)_{prof}_norm.tif").values[0]
    areiag = rxr.open_rasterio(fr"D:/Mestrado/Trabalho Final/SIG/Areia(Fina)_{prof}_norm.tif").values[0]

    areia = areiaf + areiag

    # transforma em vetores 1D
    areia_1d  = areia.ravel()
    argila_1d = argila.ravel()

    print("- Calculando as texturas dos solos")
    soil_textures = calculate_soil_type(areia_1d, argila_1d, return_index=True)
    soil_textures_2d = soil_textures.reshape(argila.shape)

    print(f"- Salvando textura para {prof}cm")

    # Pega o template de coordenadas
    coords = {"y": uso_solo.y, "x": uso_solo.x}

    da = xr.DataArray(
        soil_textures_2d,
        coords=coords,
        dims=("y","x"),
        name="texture"
    )
    da = da.expand_dims("band").assign_coords(band=[1])

    stack = da.rio.write_crs(uso_solo.rio.crs)
    stack = stack.rio.write_transform(uso_solo.rio.transform())
    stack.rio.to_raster(fr"D:/Mestrado/Trabalho Final/SIG/textura_{prof}.tif", compress="LZW")
    

    

Obtendo texturas para 2cm:
Calculando as texturas dos solos
Salvando textura para 2cm
Obtendo texturas para 20cm:
Calculando as texturas dos solos
Salvando textura para 20cm
Obtendo texturas para 80cm:
Calculando as texturas dos solos
Salvando textura para 80cm
